In [28]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from pathlib import Path
from keras import backend as K
from collections import defaultdict
from keras.models import load_model

K.set_image_data_format('channels_first')

import cv2
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import time
from fr_utils import *

np.set_printoptions(threshold=np.nan)

In [29]:
def triplet_loss(y_true, y_pred, alpha = 0.2):

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)
    
    return loss

In [30]:
Rmodel = load_model('my_model.h5', custom_objects={'triplet_loss': triplet_loss})

print("Total Params:", Rmodel.count_params())

Total Params: 3743280


In [31]:
def getAllFiles(pathDir2):
    pathDir = Path(pathDir2)
    files = [p for p in pathDir.iterdir() if p.is_file ()]
    return files;

In [32]:
#using pil to load image

def PIL2array(img):
    return np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)

min_side = 600
np_imgs = []
all_imgs = getAllFiles('images/fulltest/')
for cur_img_path in all_imgs:
    img = Image.open(cur_img_path)
    img = img.convert('RGB')
    width, height = img.size
    if height < width:
        width = int(1.0 * width * min_side / height)
        height = min_side
    else:
        height = int(1.0 * height * min_side / width)
        width = min_side
    size = width + 1, height + 1
    img = img.resize(size)
    tmp = PIL2array(img)
    np_imgs.append(tmp)


In [33]:
#using cv2 to load image that 8 times faster than pil

min_side = 600
np_imgs = []
all_imgs = getAllFiles('images/fulltest/')

for cur_img_path in all_imgs:
    tmp = cv2.imread(str(cur_img_path), 1)
    height = tmp.shape[0]
    width = tmp.shape[1]
    if height < width:
        width = int(1.0 * width * min_side / height)
        height = min_side
    else:
        height = int(1.0 * height * min_side / width)
        width = min_side

    tmp = cv2.resize(tmp,(width + 1, height + 1)) 
    np_imgs.append(tmp)

In [ ]:
dim = 200
stride = 100

mat = []
encodings = []

start_time = time.time()

for row in range(0, int(3 * min_side / 2), stride):
    for col in range(0, int(3 * min_side / 2), stride):
        mat.append([row, col])


for cur_img in np_imgs:
    tmp = []
    for r in mat:
        if cur_img.shape[0] - dim > r[0] and cur_img.shape[1] - dim > r[1]:
            tmp.append(img_to_encoding_range(dim, r[0], r[1], cur_img, Rmodel))
    tmp.append(img_to_encoding(cur_img, Rmodel))
    tmp.append(img_to_encoding_bot(cur_img, Rmodel))
    tmp.append(img_to_encoding_top(cur_img, Rmodel))
    encodings.append(tmp)

print(time.time() - start_time)

In [ ]:
def union_find(tree_size):
    for i in range(0, tree_size):
        par.append(i)

def get_par(cur_node):
    if cur_node == par[cur_node]:
        return cur_node
    else:
        par[cur_node] = get_par(par[cur_node])
        return par[cur_node]

def merge(a, b):
    a = get_par(a)
    b = get_par(b)
    par[a] = b

In [ ]:
MIN_EPS = 0.94
MAX_EPS = 1.04

def size_compare(a, b):
    ret = (1.0 * a[0] * b[1]) / (1.0 * a[1] * b[0]) 
    if ret > MIN_EPS and ret < MAX_EPS:
        return True
    else:
        return False


In [ ]:
par = []
union_find(len(encodings))
for i in range(0, len(encodings)):
    for j in range(i + 1, len(encodings)):
        if size_compare(np_imgs[i].shape, np_imgs[j].shape) == False:
            continue
        cnt = 0
        size = min(len(encodings[i]), len(encodings[j]))
        for k in range(0, size):
            dist = np.linalg.norm(encodings[i][k]-encodings[j][k])
            if dist < 0.42:
                cnt += 1
        if cnt > size / 4 + 1:
            merge(i, j)
        
d = defaultdict(list)
for i in range(0, len(encodings)):
    cur_par = get_par(i)
    d[cur_par].append(all_imgs[i])

failed = 0

for cur_par in d:
    print("----------------------------")
    print("Parent : ", cur_par)
    print("----------------------------")
    print(d[cur_par])
    if len(d[cur_par]) == 1:
        failed += 1
    #    print(d[cur_par])
print(len(d))
print(failed)